In [1]:
import mysql.connector as mysql
from mysql.connector import errorcode

ds_host = "localhost"
db_username = "root"
db_password = "qiwoqqwu"

try:
    conn = mysql.connect(
        host=ds_host,
        port=3306,
        user=db_username,
        password=db_password,
        database = "week10_store_procedure"
    )
    cursor = conn.cursor()

    # Câu lệnh tạo database
    cursor.execute("CREATE DATABASE IF NOT EXISTS week10_store_procedure")
    print(" Đã tạo (hoặc đã tồn tại) database 'week10_store_procedure'.")


except mysql.Error as err:
    print(" Lỗi:", err)


 Đã tạo (hoặc đã tồn tại) database 'week10_store_procedure'.


In [42]:
cursor.execute("DROP PROCEDURE IF EXISTS get_customer_order_count")
while cursor.nextset(): pass
conn.commit()


In [52]:
cursor.execute("DROP PROCEDURE IF EXISTS get_customer_order_count")

create_proc_1 = """
CREATE PROCEDURE get_customer_order_count(IN customer_id INT, OUT order_count INT)
BEGIN
    SELECT COUNT(*) INTO order_count
    FROM orders
    WHERE orders.customer_id = customer_id;
END
"""
cursor.execute(create_proc_1)
conn.commit()


In [79]:
def get_order_count(customer_id):
    query = """
        SELECT COUNT(*) as order_count
        FROM orders
        WHERE customer = %s
    """
    cursor.execute(query, (customer_id,))
    result = cursor.fetchone()
    order_count = result[0] if result else 0
    print(f"[Raw SQL] Tổng số đơn hàng của khách hàng {customer_id} là: {order_count}")
get_order_count(1)



[Raw SQL] Tổng số đơn hàng của khách hàng 1 là: 2


In [54]:
cursor.execute("DESCRIBE orders")
for row in cursor.fetchall():
    print(row)


('id', 'int', 'NO', 'PRI', None, '')
('customer', 'int', 'YES', 'MUL', None, '')
('order_date', 'date', 'YES', '', None, '')
('total', 'int', 'YES', '', None, '')


In [44]:
cursor.execute("DROP PROCEDURE IF EXISTS get_employee_name")

create_proc_2 = """
CREATE PROCEDURE get_employee_name(IN employee_id INT, OUT employee_name VARCHAR(255))
BEGIN
    SELECT name INTO employee_name
    FROM employees
    WHERE id = employee_id;
END
"""
cursor.execute(create_proc_2)
conn.commit()


In [58]:
def get_employee_name(employee_id):
    query = """
        SELECT name
        FROM employees
        WHERE id = %s
    """
    cursor.execute(query, (employee_id,))
    result = cursor.fetchone()
    employee_name = result[0] if result else "Không tìm thấy"
    print(f"Tên nhân viên có id {employee_id}: {employee_name}")
    return employee_name

# Gọi thử
employee_id_input = 1
employee_name = get_employee_name(employee_id_input)


Tên nhân viên có id 1: David


In [45]:
cursor.execute("DROP PROCEDURE IF EXISTS update_customer_city")

create_proc_3 = """
CREATE PROCEDURE update_customer_city(IN customer_id INT, IN new_city VARCHAR(255))
BEGIN
    UPDATE customers
    SET city = new_city
    WHERE id = customer_id;
END
"""
cursor.execute(create_proc_3)
conn.commit()


In [60]:
def update_new_city(new_city, customer_id):
    query = """
        UPDATE customers
        SET city = %s
        WHERE id = %s
    """
    cursor.execute(query, (new_city, customer_id))
    conn.commit()  # Đừng quên commit nha!

# Gọi thử
customer_id_input = 1
new_city_input = "NgheAn"
update_new_city(new_city_input, customer_id_input)

# Kiểm tra kết quả
query = "SELECT * FROM customers"
cursor.execute(query)
data = cursor.fetchall()
print(data)


[(1, 'Alice', 'NgheAn'), (2, 'Bob', 'Ho Chi Minh'), (3, 'Charlie', 'Hanoi')]


In [46]:
cursor.execute("DROP PROCEDURE IF EXISTS get_products_by_price")

create_proc_4 = """
CREATE PROCEDURE get_products_by_price(IN min_price INT)
BEGIN
    SELECT * FROM products
    WHERE price >= min_price;
END
"""
cursor.execute(create_proc_4)
conn.commit()


In [76]:
def get_products_by_price(min_price):
    cursor.callproc("get_products_by_price", (min_price,))
    for result in cursor.stored_results():
        products = result.fetchall()
        print(f"[Câu 4] Sản phẩm có giá từ {min_price} trở lên:")
        for prod in products:
            # Giả sử bảng products có cột: id, name, price
            prod_id, name, price = prod
            print(f"- {name}: {price}")


In [77]:
get_products_by_price(200)


[Câu 4] Sản phẩm có giá từ 200 trở lên:
- Laptop: 1500
- Monitor: 300


In [67]:
cursor.execute("DROP PROCEDURE IF EXISTS delete_low_total_orders")

create_proc_5 = """
CREATE PROCEDURE delete_low_total_orders(IN min_total INT)
BEGIN
    DELETE FROM order_items
    WHERE order_id IN (
        SELECT order_id FROM (
            SELECT o.id AS order_id, SUM(oi.quantity * p.price) AS total
            FROM orders o
            JOIN order_items oi ON o.id = oi.order_id
            JOIN products p ON oi.product_id = p.id
            GROUP BY o.id
            HAVING total < min_total
        ) AS low_orders
    );

    DELETE FROM orders
    WHERE id IN (
        SELECT order_id FROM (
            SELECT o.id AS order_id, SUM(oi.quantity * p.price) AS total
            FROM orders o
            JOIN order_items oi ON o.id = oi.order_id
            JOIN products p ON oi.product_id = p.id
            GROUP BY o.id
            HAVING total < min_total
        ) AS low_orders
    );
END
"""
cursor.execute(create_proc_5)
conn.commit()


In [72]:
def delete_low_total_orders(min_total):
    cursor.callproc("delete_low_total_orders", (min_total,))
    conn.commit()
    print(f"[Câu 5] Đã xoá các đơn hàng có tổng < {min_total}")


In [73]:
delete_low_total_orders(500)


[Câu 5] Đã xoá các đơn hàng có tổng < 500


In [74]:
cursor.execute("SELECT * FROM orders")
for row in cursor.fetchall():
    print(row)


(1, 1, datetime.date(2024, 1, 10), 500)
(2, 1, datetime.date(2024, 3, 1), 800)
(3, 2, datetime.date(2024, 2, 20), 1200)
(4, 3, datetime.date(2024, 1, 15), 200)


In [ ]:
def GetProductsByPrice(min_price):
    query = """
        SELECT * FROM Products WHERE price >= ?;
    """
    cursor.execute(query, (min_price,))
    return cursor.fetchall()

min_price = 300
data = GetProductsByPrice(min_price)
print(data)

In [ ]:
def delete_order(min_total):
  query = """
      DELETE FROM Orders WHERE total < ?
  """
  cursor.execute(query, (min_total,))

min_total = 350
delete_order(min_total)

query = """
  SELECT * FROM orders
"""
data = cursor.execute(query).fetchall()
print(data)